In [1]:
import symforce
symforce.set_epsilon_to_symbol()
from symforce.values import Values
import numpy as np
import cv2 as cv
import symforce.symbolic as sf
from symforce.opt.factor import Factor
from symforce.opt.optimizer import Optimizer

In [2]:
# ground truth coefficients
ar = 1.0
br = 2.0
cr = 1.0

# initial estimates
ae = 2.0
be = -1.0
ce = 5.0

# number of data points
N = 100

w_sigma = 1.0 # noise sigma
inv_sigma = 1.0 / w_sigma

rng = np.random.default_rng()

x_data = np.linspace(0, 1, N)
y_data = np.exp(ar * x_data * x_data + br * x_data + cr) + rng.normal(0, w_sigma, N)

In [3]:
def error(abc: sf.V3, x: sf.Scalar, y: sf.Scalar) -> sf.V1:
    return sf.V1(y - sf.exp(abc[0]*x*x + abc[1]*x + abc[2]))

In [4]:
initial_values = Values(
    abc = sf.V3([ae, be, ce]),
    xdata = x_data.tolist(),
    ydata = y_data.tolist(),
    epsilon = sf.numeric_epsilon,
)

In [5]:
factors = []
for i in range(len(x_data)):
    factors.append(Factor(
        residual=error,
        keys=["abc", f"xdata[{i}]", f"ydata[{i}]"],
    ))

In [6]:
optimizer = Optimizer(
    factors=factors,
    optimized_keys=["abc"],
    debug_stats=True,
    params=Optimizer.Params(iterations=1000)
)
result = optimizer.optimize(initial_values)
result.optimized_values.get('abc')

RuntimeError: SYM_ASSERT: linearized_factor->index.tangent_dim == linearized_factor->jacobian.cols()
    --> void sym::Factor<ScalarType>::Linearize(const sym::Values<Scalar>&, sym::Factor<ScalarType>::LinearizedDenseFactor*, const std::vector<sym::index_entry_t>*) const [with ScalarType = double; sym::Factor<ScalarType>::LinearizedDenseFactor = sym::linearized_dense_factor_t]
    --> /project/symforce/opt/factor.cc:102


In [ ]:
a, b, c = result.optimized_values.get('abc')
y_cal = np.exp(a * x_data * x_data + b * x_data + c)
y_gt = np.exp(ar * x_data * x_data + br * x_data + cr)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib qt
plt.plot(x_data, y_data, '.')
plt.plot(x_data, y_cal)
plt.plot(x_data, y_gt)
plt.show()